# S12N Partner Mapping

In [0]:
DROP TABLE IF EXISTS dev.bi.s12n_partner_mapping_mgtv;
CREATE TABLE dev.bi.s12n_partner_mapping_mgtv AS
(
  WITH base_mapping AS 
(
  SELECT 
    base_phoenix_specialization_id,
    phoenix_specialization_id
  FROM
    prod.gold.phoenix_specializations_vw
  GROUP BY 1, 2
),

s12n_partner_mapping AS
(
  SELECT 
    psp.partner_id,
    p.partner_slug,
    bm.base_phoenix_specialization_id
  FROM
    prod.gold.phoenix_specializations_partners_vw psp
  JOIN
    base_mapping bm
  ON
    bm.phoenix_specialization_id = psp.phoenix_specialization_id
  JOIN
    prod.gold.partner_vw p
  ON
    psp.partner_id = p.partner_id
  WHERE
    NOT psp.is_test_specialization
)

SELECT
    partner_id,
    partner_slug,
    base_phoenix_specialization_id  
FROM 
  s12n_partner_mapping
GROUP BY 1, 2, 3
)



num_affected_rows,num_inserted_rows


# S12N Partner Average Revenue

In [0]:
DROP TABLE IF EXISTS dev.bi.s12n_partner_average_annual_b2c_revenue_mgtv;
CREATE TABLE dev.bi.s12n_partner_average_annual_b2c_revenue_mgtv AS 
(
WITH partner_revenue AS (
  -- First, join the tables and calculate yearly revenue per specialization
  SELECT 
    pm.partner_id,
    pm.partner_slug,
    rev.base_phoenix_specialization_id,
    rev.launch_date,
    SUM(rev.b2c_revenue) / 
      GREATEST(
        DATEDIFF(CURRENT_DATE(), launch_date) / 365.0,
        0.1  -- Use 1 if less than a year has elapsed to avoid division by small numbers
      ) as yearly_revenue
  FROM dev.bi.s12n_partner_mapping_mgtv pm
  JOIN dev.bi.s12n_estimated_total_b2c_revenue_mgtf rev
  -- JOIN dev.bi.s12n_estimated_total_b2c_revenue_mgtv rev 
    ON pm.base_phoenix_specialization_id = rev.base_phoenix_specialization_id
  GROUP BY 
    pm.partner_id,
    pm.partner_slug,
    rev.base_phoenix_specialization_id,
    rev.launch_date
)

SELECT 
  partner_id,
  partner_slug,
  COUNT(DISTINCT base_phoenix_specialization_id) as number_of_specializations,
  AVG(yearly_revenue) as avg_yearly_revenue_across_s12ns,
  avg_yearly_revenue_across_s12ns/NULLIF(number_of_specializations, 0) as avg_yearly_revenue_per_s12n
FROM partner_revenue
GROUP BY 
  partner_id,
  partner_slug
)

num_affected_rows,num_inserted_rows


# QA

In [0]:
select * from dev.bi.s12n_partner_average_annual_b2c_revenue_mgtv order by avg_yearly_revenue_across_s12ns desc

partner_id,partner_slug,number_of_specializations,avg_yearly_revenue_across_s12ns,avg_yearly_revenue_per_s12n
718,intuit,1,3186378.9915570000,3186378.9915570000
1442,hrci,1,1960617.8171910000,1960617.8171910000
463,google,89,1905826.8214789551,21413.7845109995
1,stanford,6,1581230.2899481667,263538.3816580278
475,deeplearning-ai,15,1285308.8640860000,85687.2576057333
875,tableau,1,1169192.7333080000,1169192.7333080000
496,adobe,3,773962.8322756667,257987.6107585556
1571,epic-games,1,699876.9428170000,699876.9428170000
532,amazon,1,528505.8192760000,528505.8192760000
705,microsoft,41,524202.5290983659,12785.4275389845
